Importando as bibliotecas utilizadas ... 

In [154]:
import numpy as np
import pandas as pd
import pickle
import gensim
import spacy

Inicializando o DataFrame da matriz dos conhecimentos, 
utilizando a função 'fillna' para consertar as células mescladas,
lendo a partir da linha 4
e lendo até a coluna 9 (excluso)

In [155]:
crea_df = pd.read_excel("Matriz_do_Conhecimento.xls", skiprows=4).iloc[:, :9].fillna(method="ffill")

Corrigindo valores que foram incorretamente preenchido pelo método fillna("ffill"):

In [156]:
crea_df = crea_df.replace({"TÓPICOS": np.NaN, "Nº DE ORDEM DOS TÓPICOS": np.NaN})

Salvando o Dataframe com o pickle

In [157]:
with open('crea_df_pickle.txt', 'wb') as f:
    pickle.dump(crea_df, f)

Abrindo o dataFrame em formato Binário e Imprimindo o DataFrame para fins de visualização:

In [158]:
with open('crea_df_pickle.txt', 'rb') as f:
    crea_df = pickle.load(f)

# seção de Construção de Edificações
subsection1_df = crea_df[ crea_df['SUB-SETOR'] == 'Construção de Edificações']

Importando as matérias que foram manualmente obtidas e salvas em um arquivo json

In [159]:
subjects_df = pd.read_json("MechatronicsEngeneeringSubjects.json")
subjects_df

,codigo,nome,ementa,conteudo
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,Princípios fundamentais de construção de progr...,
1,IFD0171,FISICA 1,Módulos 1: Unidades e grandezas físicas 2: Vet...,"I-MEDICAO: GRANDEZAS, PADROES E UNIDADES FISIC..."
2,IFD0173,FISICA 1 EXPERIMENTAL,MEDIDAS E ERROS. ANALISE GRAFICA. ATRITO. COLI...,I-CLASSIFICACAO DOS ERROS. CALCULO DE ERRO EXP...
3,MAT0025,CÁLCULO 1,"Funções de uma variável real, limite e continu...",1. Funções: conceito de função exemplo de funç...
4,ENM0133,INTRODUCAO A ENGENHARIA MECATRONICA,"A Universidade de Brasília. Engenharia, Contro...",
5,IQD0125,QUIMICA GERAL TEORICA,Abordagem conceitual dos princípios fundamenta...,1.Estrutura Atômica e a Lei Periódica: O Model...
6,IQD0126,QUIMICA GERAL EXPERIMENTAL,CARACTERIZACAO DA NATUREZA E DO PAPEL DAS INVE...,1) Noções Básicas sobre Segurança no Trabalho ...
7,ENM0131,DESENHO MECANICO ASSISTIDO POR COMPUTADOR 1,Objetivos da Disciplina: A disciplina objetiva...,- Normas de desenho Técnico- Introdução ao CAD...
8,EST0023,PROBABILIDADE E ESTATÍSTICA,"Análise Descritiva, Cálculo de Probabilidades,...",UNIDADE I - Análise Descritiva de Dados - Popu...
9,IFD0175,FISICA 2,Módulos 1: Equilíbrio e elasticidade 2: Gravit...,1-CONTEUDO TEORICO 1-DINAMICA DA ROTACAO 1.1- ...


Criando uma função de pre-processamento de texto. Retira ruidos(cleaning) -> tokeniza-lemmatiza -> depois retira stopwords;

Recebe um texto no formato de string e retorna uma lista de strings com as palavras do documento.

Todo: A lemmatização causou alguns conflitos, como "serie" -> "seriar" <- "seriam". Talvez seja melhor não usar os lemmas?(pelo menos nos stopwords). Adicionar Bigramas e Trigramas

In [183]:
from nltk.corpus import stopwords

nlp = spacy.load('pt_core_news_lg')

# TODO: find out if it is definately impossible to disable PoS tags in nlp model; alternative: convert all verbs to noun using wordnet and .pos from spacy?
# TODO: implementar o tagger utilizando o seguinte banco de dados: https://www.nltk.org/howto/portuguese_en.html#accessing-the-macmorpho-tagged-corpus
# tagger = nlp.get_pipe("tagger")
# doc = nlp("eletromagnetismo serie")
# print(tagger.model.predict([doc])[0][1])
# print(tagger.labels)

# * adding custom texts that dont represent real words
noises_list = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x", "xi"]

stopWords_list = stopwords.words("portuguese")

# * adding custom words to StopWords list
stopWords_list += [
    'referente',
    'seguinte'
]

# * preprocessing stopwords to correct format
stopWords_list = gensim.utils.simple_preprocess(" ".join(stopWords_list), deacc=True, min_len=1, max_len=40)

# print(stopWords_list)

# * manual intervention, changing final lemmas
intervention_dict = {
    "campar": "campo",
    "seriar":"serie",
    "eletromagnetico":"eletromagnetismo",
}

def preprocess(text):
    # * importing stopwords from nltk and spacy pipeline
    global nlp
    global stopWords_list
    global noises_list
    global intervention_dict

    # * preprocessing text with gensim.simple_preprocess, eliminating noises: lowercase, tokenized, no symbols, no numbers, no accents marks(normatize)
    text_list = gensim.utils.simple_preprocess(text, deacc=True, min_len=1, max_len=40)

    # * recombining tokens to a string type object and removing remaining noises
    text_str = " ".join([word for word in text_list if word not in noises_list])

    # * preprocessing with spacy, retokenizing -> tagging parts of speech (PoS) -> parsing (assigning dependencies between words) -> lemmatizing
    text_doc = nlp(text_str)

    # * re-tokenization, removing stopwords and lemmatizing
    lemmatized_text_list = [token.lemma_ for token in text_doc if token.text not in stopWords_list]

    # * manual intervention conversion of lemmas
    output = []
    for token in lemmatized_text_list:
        if token in intervention_dict:
            output.append(intervention_dict[token])
        else:
            output.append(token)
            
    return output

Pre-processando as matérias com Stopwords do NLTK e função do Gensim. E adicionando uma nova columa ao Dataframe que consiste em todo o texto da matéria

In [184]:
documents_list = []


for i, row in subjects_df.iterrows():

    # * reading values of each subject (row)
    subject_id = row["codigo"]
    name = row["nome"]
    syllabus = row["ementa"]
    content = row["conteudo"]

    # * combining them to create the subject document
    text = name + ' ' + syllabus + ' ' + content
    
    # * preprocessing
    preProcessedText = preprocess(text)
    documents_list.append(preProcessedText)

documents_series = pd.Series(documents_list, name="documento")

documents_df = pd.concat([subjects_df, documents_series], axis=1)

Salvando o Dataframe das matérias,  reabrindo-o e imprimindo o DataFrame para fins de visualização:

In [185]:
with open('documents_df.txt', 'wb') as f:
    pickle.dump(documents_df, f)

In [186]:
with open('documents_df.txt', 'rb') as f:
    documents_df = pickle.load(f)

documents_df

,codigo,nome,ementa,conteudo,documento
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,Princípios fundamentais de construção de progr...,,"[algoritmo, programacao, computador, principio..."
1,IFD0171,FISICA 1,Módulos 1: Unidades e grandezas físicas 2: Vet...,"I-MEDICAO: GRANDEZAS, PADROES E UNIDADES FISIC...","[fisica, modulos, unidade, grandeza, fisicas, ..."
2,IFD0173,FISICA 1 EXPERIMENTAL,MEDIDAS E ERROS. ANALISE GRAFICA. ATRITO. COLI...,I-CLASSIFICACAO DOS ERROS. CALCULO DE ERRO EXP...,"[fisica, experimental, medir, erro, analisar, ..."
3,MAT0025,CÁLCULO 1,"Funções de uma variável real, limite e continu...",1. Funções: conceito de função exemplo de funç...,"[calcular, funcoes, variavel, real, limitar, c..."
4,ENM0133,INTRODUCAO A ENGENHARIA MECATRONICA,"A Universidade de Brasília. Engenharia, Contro...",,"[introducao, engenhar, mecatronica, universida..."
5,IQD0125,QUIMICA GERAL TEORICA,Abordagem conceitual dos princípios fundamenta...,1.Estrutura Atômica e a Lei Periódica: O Model...,"[quimica, geral, teorica, abordagem, conceitua..."
6,IQD0126,QUIMICA GERAL EXPERIMENTAL,CARACTERIZACAO DA NATUREZA E DO PAPEL DAS INVE...,1) Noções Básicas sobre Segurança no Trabalho ...,"[quimica, geral, experimental, caracterizacao,..."
7,ENM0131,DESENHO MECANICO ASSISTIDO POR COMPUTADOR 1,Objetivos da Disciplina: A disciplina objetiva...,- Normas de desenho Técnico- Introdução ao CAD...,"[desenhar, mecanico, assistir, computador, obj..."
8,EST0023,PROBABILIDADE E ESTATÍSTICA,"Análise Descritiva, Cálculo de Probabilidades,...",UNIDADE I - Análise Descritiva de Dados - Popu...,"[probabilidade, estatistica, analisar, descrit..."
9,IFD0175,FISICA 2,Módulos 1: Equilíbrio e elasticidade 2: Gravit...,1-CONTEUDO TEORICO 1-DINAMICA DA ROTACAO 1.1- ...,"[fisica, modulos, equilibrio, elasticidade, gr..."


Salvando a coluna de documentos para tornar legível e facilitar a busca, assim podendo conferir se os lemmas estão satisfatórios e se condizem

In [176]:
with open("debbuging_docs.txt", "w+") as f:
    f.write( documents_df['documento'].to_json())

construindo o Corpus, vetorizando os documentos com o bag-of-words do gensim

In [165]:
from gensim import corpora

lemmatizedData = documents_df["documento"].tolist()

# * gensim dictionary object, which will track each word to its respective id
id2wordDict = corpora.Dictionary(lemmatizedData)

# * gensim doc2bow method to map each word to a integer id and its respective frequency
corpus = [id2wordDict.doc2bow(text) for text in lemmatizedData]

# * list of list of tuples (id of a word, frequency)
corpus

[[(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 3),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 3),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 3),
  (39, 3),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 1)],
 [(35, 1),
  (37, 1),
  (45, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 3),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 2),
  (61, 1),
  (62, 5),
  (63, 2),
  (64, 3),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 3),
  (69, 1),
  (70, 5),
  (71, 1),
  (72, 6),
  (73, 1),
  (74, 1),
  (75, 2),
  (76, 4),
  (77, 1),
  (78, 1),
  (79, 3),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 10),
  (84, 1),
  (85, 1),
  (86, 3),
  (87, 

Utilizando tf-idf do gensim para descobrir as palavras mais importantes e dar mais pesos a elas;

tupla(int, int) -> tupla(int, float)

td-idf(term_i, document_j) = freq(i,j) * log2 ( inverse_document_frequency(i) )

freq(i, j) = total occurances of i in j / total words in j

inverse_document_frequency(i) = total documents / documents that have at least one occurance of i 

O parâmetro id2word recebe o dicionário que mapeia as palavras com os respectivos IDs.

In [166]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word=id2wordDict)

LSI, pega a matriz do corpus e a decompõe utilizando o SVD. Das três matrizes criadas, utiliza-se apenas o right singular vectors que representa a relação entre os tópicos latentes com as palavras.

O parâmetro id2word recebe o dicionário que mapeia as palavras com os respectivos IDs. E o parâmetro power_iters define o número de iterações para treinamento do modelo, e, portanto, quanto maior o valor, mais acurado e mais devagar vai ser o treino modelo

In [167]:
lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=id2wordDict, num_topics=100, power_iters=10)

lsi_model.print_topics()

[(0,
  '-0.251*"circuito" + -0.197*"transformar" + -0.184*"analisar" + -0.141*"fourier" + -0.137*"seriar" + -0.135*"laplace" + -0.134*"movimentar" + -0.125*"energia" + -0.123*"tecnicas" + -0.120*"senoidal"'),
 (1,
  '-0.267*"circuito" + 0.228*"movimentar" + 0.204*"rigidos" + 0.196*"corpo" + 0.178*"momento" + -0.175*"transformar" + 0.172*"energia" + -0.145*"analisar" + -0.136*"senoidal" + -0.129*"tecnicas"'),
 (2,
  '0.241*"transformar" + 0.203*"fourier" + -0.193*"conversor" + -0.193*"combinacionais" + -0.193*"sequencial" + -0.181*"circuito" + 0.178*"seriar" + 0.173*"laplace" + -0.144*"projeto" + 0.132*"funcoes"'),
 (3,
  '-0.200*"senoidal" + -0.177*"permanente" + -0.173*"regime" + -0.157*"analisar" + 0.153*"funcoes" + 0.144*"combinacionais" + 0.144*"conversor" + 0.144*"sequencial" + 0.129*"integrar" + 0.129*"derivar"'),
 (4,
  '-0.272*"c" + -0.244*"motor" + 0.211*"transformar" + -0.177*"gerador" + -0.146*"magnetico" + -0.139*"corrente" + 0.138*"rede" + -0.137*"tensao" + 0.133*"laplace"

left singular vector -> term-to-topic matrix (não será utilizado)

In [168]:
# * U Matrix
print(np.shape(lsi_model.projection.u))
lsi_model.projection.u

(1544, 41)


array([[-0.00641925,  0.00060278, -0.00381761, ..., -0.00042425,
         0.00172134, -0.0025063 ],
       [-0.00641925,  0.00060278, -0.00381761, ..., -0.00042425,
         0.00172134, -0.0025063 ],
       [-0.02540838, -0.02520003, -0.06103359, ...,  0.00155577,
         0.01165526,  0.00071098],
       ...,
       [-0.00414332,  0.00309265, -0.00737406, ...,  0.0002754 ,
        -0.00045605, -0.00043473],
       [-0.00414332,  0.00309265, -0.00737406, ...,  0.0002754 ,
        -0.00045605, -0.00043473],
       [-0.00414332,  0.00309265, -0.00737406, ...,  0.0002754 ,
        -0.00045605, -0.00043473]])

singular values -> "impacto" (abrangência?) de cada tópico

Pode-se escolher um valor de corte para reduzir o valor do parametro num_topics, aumentando, assim, o desempenho de tempo. O número de tópicos latentes não tem como ser maior que o numero de documentos

In [169]:
print("número de documentos/ementas/matérias:", len(subjects_df))
print("topicos latentes encontrados com um \'impacto\' alto:", len(lsi_model.projection.s))

# * S matrix (sigma)
print(np.shape(lsi_model.projection.s))
lsi_model.projection.s

número de documentos/ementas/matérias: 41
topicos latentes encontrados com um 'impacto' alto: 41
(41,)


array([1.64966956, 1.46203642, 1.35084967, 1.28890539, 1.24580017,
       1.21309217, 1.1739015 , 1.15533653, 1.1274372 , 1.11380519,
       1.09560611, 1.07907601, 1.05441718, 1.04836082, 1.03642657,
       1.010186  , 1.00369415, 0.99734939, 0.9925719 , 0.97971303,
       0.97133401, 0.95664393, 0.9503746 , 0.94562274, 0.93571345,
       0.90987014, 0.88729319, 0.87001941, 0.86214282, 0.84285669,
       0.84074217, 0.81317102, 0.80490511, 0.79917871, 0.76496516,
       0.73837239, 0.7280023 , 0.6964949 , 0.53210066, 0.42816055,
       0.40674317])

right singular vectors -> document-to-topic matrix (Não é diretamente calculado e armazenado pois pode ser muito grande devido a quantidade de documentos -> num_topic x documents)

Será o documento armazenado e utilizado para fazer as queries de similaridade

In [170]:
V_matrix = gensim.matutils.corpus2dense(lsi_model[tfidf_model[corpus]], len(lsi_model.projection.s)).T / lsi_model.projection.s
print(np.shape(V_matrix))
# * V or V^T matrix
# * representado por lsi_model[x]
V_matrix

(41, 41)


array([[-0.07356558,  0.00612226, -0.03582548, ..., -0.00156823,
         0.00511996, -0.00708186],
       [-0.24064223,  0.35342847, -0.10553482, ...,  0.02580299,
        -0.02323966,  0.02978337],
       [-0.17137737,  0.25324323, -0.05997425, ..., -0.0018203 ,
         0.00529626, -0.00256913],
       ...,
       [-0.09012535, -0.08294869, -0.08494767, ..., -0.05679201,
         0.00734282,  0.00338938],
       [-0.11957763, -0.03858938, -0.00544411, ...,  0.07753503,
         0.02257421, -0.03106496],
       [-0.02739007,  0.01811909, -0.0399173 , ...,  0.00058722,
        -0.00078247, -0.00070857]])

Salvando, então, o corpus já processado pelo tf_idf e também a matrix V de LSI em um arquivo texto do tipo Matrix Market format, que permite que a matriz seja guardado em um arquivo texto, mas também pode ser utilizado para cálculos. (o método Serialize guardas os indexes)

In [171]:
gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('lsi_model_mm', lsi_model[tfidf_model[corpus]])

Abrindo as matrizes salvas nos arquivos textos

In [172]:
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

# * features of td-idf matrix are the different words
print(tfidf_corpus)

# * features of LSI model are the latent topics discovered
print(lsi_corpus)

MmCorpus(41 documents, 1544 features, 3021 non-zero entries)
MmCorpus(41 documents, 41 features, 1681 non-zero entries)


Buscas de Similaridade Semântica utilizando o método Matrix similarity do gensim, o qual computa o "cosine similarity".

Se forem buscados termos que não foram adicionados ao dicionário id2wordDict, não há como mapear esse documento

O parâmetro num_best define quantos dos documentos mais proximos será buscado

In [173]:
from gensim.similarities.docsim import MatrixSimilarity

cosineSimilarity = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms, num_best=8)

def search_similarity_query(search_document):

    # * preprocessing and processing until becomes a matrix of type term_to_topic (V)
    doc = preprocess(search_document)
    query_bow = id2wordDict.doc2bow(doc)
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    # * cossine similarity between the vector of the new document vs all other vectors of documents
    # * returns a list of tuples (id of compared document, similarity)
    ranking = cosineSimilarity[query_lsi]

    ranking.sort(key=lambda unit: unit[1], reverse= True)
    result = []

    for subject in ranking:

        result.append (
            {
                'Relevancia': round((subject[1] * 100),6),
                'Código da Matéria': subjects_df['codigo'][subject[0]],
                'Nome da matéria': subjects_df['nome'][subject[0]]
            }

        )
    
    output = pd.DataFrame(result, columns=['Relevancia','Código da Matéria','Nome da matéria'])
    if len(output) == 0:
        return "O texto procurado não há similaridade com nenhuma das demais"
    return output



Utilizando documentos no formato de str, pode-se realizar buscas de similaridade semântica entre as matérias

In [180]:
search_similarity_query("eletromagnético")
print(preprocess("eletromagnético"))

['eletromagnetico']


Multi-queries:

In [175]:
subsection1_df.head(15)

for i in range(15):
    text = subsection1_df.iloc[i, 8]
    print(f'texto buscado: {text}')
    print(f'texto buscado preprocessado: {preprocess(text)}')
    print( search_similarity_query(text), end='\n\n')


texto buscado: Cálculo diferencial e integral 
texto buscado preprocessado: ['calcular', 'diferencial', 'integral']
   Relevancia Código da Matéria        Nome da matéria
0   94.676280           MAT0025              CÁLCULO 1
1   29.593253           MAT0027              CÁLCULO 3
2   29.400170           MAT0028    VARIAVEL COMPLEXA 1
3   21.387300           ENE0045             ELETRÔNICA
4   15.841059           MAT0026              CÁLCULO 2
5   13.827941           ENE0037  CIRCUITOS ELÉTRICOS 2
6   10.041592           IFD0179               FISICA 3
7    9.956031           IFD0177  FISICA 2 EXPERIMENTAL

texto buscado: Cálculo numérico 
texto buscado preprocessado: ['calcular', 'numerico']
   Relevancia Código da Matéria                  Nome da matéria
0   83.766907           ENM0027  TECNOLOGIAS DE COMANDO NUMERICO
1   38.391241           MAT0053                 CALCULO NUMERICO
2   23.907737           MAT0027                        CÁLCULO 3
3   20.602205           MAT0025          